In [6]:
import pandas as pd;
import numpy as np;
import json
key = open("keys.py").readline().replace("\n","")

In [15]:
########### Python 3.2 #############
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'api_key': key,
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('api.wmata.com')
    conn.request("GET", "/Rail.svc/json/jLines?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read().decode('utf-8')
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

####################################

{"Lines":[{"LineCode":"BL","DisplayName":"Blue","StartStationCode":"J03","EndStationCode":"G05","InternalDestination1":"","InternalDestination2":""},{"LineCode":"GR","DisplayName":"Green","StartStationCode":"F11","EndStationCode":"E10","InternalDestination1":"","InternalDestination2":""},{"LineCode":"OR","DisplayName":"Orange","StartStationCode":"K08","EndStationCode":"D13","InternalDestination1":"","InternalDestination2":""},{"LineCode":"RD","DisplayName":"Red","StartStationCode":"A15","EndStationCode":"B11","InternalDestination1":"A11","InternalDestination2":"B08"},{"LineCode":"SV","DisplayName":"Silver","StartStationCode":"N06","EndStationCode":"G05","InternalDestination1":"","InternalDestination2":""},{"LineCode":"YL","DisplayName":"Yellow","StartStationCode":"C15","EndStationCode":"E06","InternalDestination1":"E01","InternalDestination2":""}]}


In [19]:
result = json.loads(data)
# print (json.dumps(result,indent=4))

{
    "Lines": [
        {
            "EndStationCode": "G05",
            "LineCode": "BL",
            "InternalDestination2": "",
            "DisplayName": "Blue",
            "InternalDestination1": "",
            "StartStationCode": "J03"
        },
        {
            "EndStationCode": "E10",
            "LineCode": "GR",
            "InternalDestination2": "",
            "DisplayName": "Green",
            "InternalDestination1": "",
            "StartStationCode": "F11"
        },
        {
            "EndStationCode": "D13",
            "LineCode": "OR",
            "InternalDestination2": "",
            "DisplayName": "Orange",
            "InternalDestination1": "",
            "StartStationCode": "K08"
        },
        {
            "EndStationCode": "B11",
            "LineCode": "RD",
            "InternalDestination2": "B08",
            "DisplayName": "Red",
            "InternalDestination1": "A11",
            "StartStationCode": "A15"
        },
        {
 

In [34]:
paths = {}
master_paths = pd.DataFrame();
for i in result['Lines']:
    params = urllib.parse.urlencode({
        # Request parameters
        'FromStationCode': i['StartStationCode'],
        'ToStationCode': i['EndStationCode'],
    })

    try:
        conn = http.client.HTTPSConnection('api.wmata.com')
        conn.request("GET", "/Rail.svc/json/jPath?%s" % params, "{body}", headers)
        response = conn.getresponse()
        data = response.read().decode('utf-8')
        print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

    ####################################
    paths[i['DisplayName']] = pd.DataFrame(json.loads(data)['Path'])
    master_paths.append(json.loads(data)['Path'])

{"Path":[{"LineCode":"BL","StationCode":"J03","StationName":"Franconia-Springfield","SeqNum":1,"DistanceToPrev":0},{"LineCode":"BL","StationCode":"J02","StationName":"Van Dorn Street","SeqNum":2,"DistanceToPrev":18695},{"LineCode":"BL","StationCode":"C13","StationName":"King St-Old Town","SeqNum":3,"DistanceToPrev":20246},{"LineCode":"BL","StationCode":"C12","StationName":"Braddock Road","SeqNum":4,"DistanceToPrev":3453},{"LineCode":"BL","StationCode":"C10","StationName":"Ronald Reagan Washington National Airport","SeqNum":5,"DistanceToPrev":16108},{"LineCode":"BL","StationCode":"C09","StationName":"Crystal City","SeqNum":6,"DistanceToPrev":2939},{"LineCode":"BL","StationCode":"C08","StationName":"Pentagon City","SeqNum":7,"DistanceToPrev":4068},{"LineCode":"BL","StationCode":"C07","StationName":"Pentagon","SeqNum":8,"DistanceToPrev":3216},{"LineCode":"BL","StationCode":"C06","StationName":"Arlington Cemetery","SeqNum":9,"DistanceToPrev":7057},{"LineCode":"BL","StationCode":"C05","Stat

In [74]:
station_names = []
paths.keys()
for line in paths.keys():
    for station in paths[line]['StationName']:
        station_names.append(station);
    

In [79]:
station_names.count("Metro Center")

4

In [152]:
def findLines(station):
    stations = [];
    for line in paths.keys():
        for stat in paths[line]['StationName']:
            if stat==station:
                stat_color = paths[line][paths[line]['StationName']==stat]['LineCode'];
                index = str(stat_color).index("    ")+4;
#                 print (str(stat_color)[index:index+2])
                stations.append((str(stat_color)[index:index+2]))
    return stations;

In [155]:
# Which lines are destination on?
destination = "Braddock Road";
start = "U Street/African-Amer Civil War Memorial/Cardozo";
findLines(destination)

['YL', 'BL']

In [157]:
findLines(start)

['YL', 'GR']

In [175]:
# Need adjacency matrix demonstrating which stations are on which lines
# e.g. [X, 2, 1]
#      [2, X, 0]
#      [1, 0, X]

# def constructAdjacencyMatrix():

yellow_line = paths['Yellow']

yellow_line[yellow_line['StationName']==start].index.tolist()
    

[13]

In [176]:
yellow_line[yellow_line['StationName']==destination].index.tolist()

[3]

In [188]:
station_names[56]

'McPherson Square'

In [192]:
findLines(station_names[56])

['OR', 'SV', 'BL']

In [197]:
shortcode_map = {"RD":"Red", "YL":"Yellow", "BL":"Blue", "GR":"Green", "SV":"Silver", "OR":"Orange"}

In [232]:
adj_matrix = {};

for n in station_names:
    current_station = {};
    for k in findLines(n):
        for z in paths[shortcode_map[k]]['StationName']:
    #         Get distance from n to z via line k
            indexOfZ = (paths[shortcode_map[k]][paths[shortcode_map[k]]['StationName']==z].index.tolist()[0])
            indexOfN = (paths[shortcode_map[k]][paths[shortcode_map[k]]['StationName']==n].index.tolist()[0])
            current_station[z]=abs(indexOfZ - indexOfN);
    adj_matrix[n] = current_station

In [233]:
adj_matrix['Rosslyn']

{'Addison Road-Seat Pleasant': 15,
 'Arlington Cemetery': 1,
 'Ballston-MU': 4,
 'Benning Road': 13,
 'Braddock Road': 6,
 'Capitol Heights': 14,
 'Capitol South': 9,
 'Cheverly': 15,
 'Clarendon': 2,
 'Court House': 1,
 'Crystal City': 4,
 'Deanwood': 14,
 'Dunn Loring-Merrifield': 7,
 'East Falls Church': 5,
 'Eastern Market': 10,
 'Farragut West': 2,
 'Federal Center SW': 8,
 'Federal Triangle': 5,
 'Foggy Bottom-GWU': 1,
 'Franconia-Springfield': 9,
 'Greensboro': 8,
 'King St-Old Town': 7,
 "L'Enfant Plaza": 7,
 'Landover': 16,
 'Largo Town Center': 17,
 'McLean': 6,
 'McPherson Square': 3,
 'Metro Center': 4,
 'Minnesota Ave': 13,
 'Morgan Boulevard': 16,
 'New Carrollton': 17,
 'Pentagon': 2,
 'Pentagon City': 3,
 'Potomac Ave': 11,
 'Ronald Reagan Washington National Airport': 5,
 'Rosslyn': 0,
 'Smithsonian': 6,
 'Spring Hill': 9,
 'Stadium-Armory': 12,
 'Tysons Corner': 7,
 'Van Dorn Street': 8,
 'Vienna/Fairfax-GMU': 8,
 'Virginia Square-GMU': 3,
 'West Falls Church-VT/UVA':

,DistanceToPrev,LineCode,SeqNum,StationCode,StationName
0,0,OR,1,K08,Vienna/Fairfax-GMU
